In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import pandas as pd
import seaborn as sns
from darts import TimeSeries
from darts.models.forecasting.linear_regression_model import LinearRegressionModel

from probafcst.plotting import plot_quantiles

sns.set_theme(style="ticks")

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

In [ ]:
y = pd.read_parquet("../data/energy.parquet")
y = TimeSeries.from_dataframe(y).drop_before(pd.Timestamp("2022"))
len(y)

In [ ]:
add_encoders = {"cyclic": {"future": ["hour", "day_of_week", "month"]}}

In [ ]:
DAY_DURATION = 24

lags = [-DAY_DURATION * i for i in [1, 7]]
lags_future_covariates = [-DAY_DURATION * i for i in [-7, -1, 0, 1, 7]]
lags, lags_future_covariates

In [ ]:
model = LinearRegressionModel(
    lags=lags,
    lags_future_covariates=lags_future_covariates,
    add_encoders=add_encoders,
    # likelihood="quantile",
    # quantiles=quantiles,
    output_chunk_length=24,
    multi_models=False,
    # solver="highs-ipm",
)
model.fit(y)

In [ ]:
y[-24 * 7 :].plot()
model.predict(24 * 7).plot()

## SKTIME

In [ ]:
# y = pd.read_parquet("../data/energy.parquet").asfreq("h")
y = pd.read_parquet("../data/bikes.parquet").asfreq("D")
DAY_DURATION = 1
freq = "D"

In [ ]:
from probafcst.models.darts import get_quantile_regressor

forecaster = get_quantile_regressor(quantiles, freq=freq, output_chunk_length=7)
forecaster

In [ ]:
from probafcst.backtest import backtest, get_window_params

window_params = get_window_params(
    n_years_initial_window=3,
    step_length_days=30,
    forecast_steps_days=7,
    freq=freq,
)
window_params

In [ ]:
results, metrics, predictions, add_metrics = backtest(
    forecaster,
    y,
    **window_params,
    quantiles=quantiles,
    backend="loky",
    splitter_type="sliding",
)

In [ ]:
add_metrics

In [ ]:
# visualize some forecasts
idx = [0, len(results) // 2, -1]
for i, (_, row) in enumerate(predictions.iloc[idx].iterrows()):
    fig, _ = plot_quantiles(row.y_test, row.y_pred_quantiles)